In [1]:
import pandas as pd
import numpy as np


In [2]:
# constants
drone_speed = 60 # km/hr

In [3]:
import os
os.getcwd()
%cd C:\Users\asus\Dropbox (Personal)\Python
os.getcwd()


[WinError 3] The system cannot find the path specified: 'C:\\Users\\asus\\Dropbox (Personal)\\Python'
C:\Users\manga


'C:\\Users\\manga'

In [4]:
import chardet   
sheet_to_df_map = pd.read_excel(r"C:\Users\manga\Drone-Data_Solutions_Operations-Research-1.xlsx", sheet_name=None) # this is the
# correct way to load the file. "r" at the beginning converts normal string it converts normal string to raw string
known_parameter_values = sheet_to_df_map['Known Parameter Values']
distances = sheet_to_df_map['Distances_values only']
demand = sheet_to_df_map['Blood Demand Data']
output = sheet_to_df_map['Output']

In [5]:
final_output_columns = output.columns
cols_to_delete = ['Step', 'Drone travel time/ minutes', 'Drone travel cost', 'Ambulance travel time', 'Ambulance travel cost']
final_output_columns = [item for item in final_output_columns if item not in cols_to_delete]
final_output = pd.DataFrame(columns = final_output_columns)



In [6]:
distance_matrix = distances.set_index('Unnamed: 0').unstack().reset_index().dropna()
distance_matrix_array = distance_matrix.values
distance_matrix.head()

distance_dict = {}
for item in distance_matrix_array:
    key = (item[1], item[0])  # columns is the starting point in the matt
    distance_dict[key] = item[2]

In [7]:
drone_current_city = "H1"
ambulance_current_city ="H1"
demand.loc[demand['Notation'] == ambulance_current_city, "Status of demand ( 1- met, 0- unmet)"] = 1  # meet demand of H1

print(f'Meeting demand of the starting city, {ambulance_current_city}')
print("***")


Meeting demand of the starting city, H1
***


In [12]:
row = {
 'Drone start': drone_current_city,
 'Drone End': drone_current_city,
 'Did Drone Deliver': False,
 'Ambulance start': ambulance_current_city,
 'Ambulance End':ambulance_current_city,
 'Did ambulance deliver': True,
  'Drone Distance': 0,
   'Ambulance Distance': 0}
final_output = final_output.append(row, ignore_index = True)

C:\Users\manga\AppData\Local\Temp\ipykernel_25240\1330095337.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_output = final_output.append(row, ignore_index = True)


In [13]:
final_output

,Drone start,Drone End,Did Drone Deliver,Ambulance start,Ambulance End,Did ambulance deliver,Ambulance Distance,Drone Distance
0,H1,H1,False,H1,H1,True,0.0,0.0
1,H1,H1,False,H1,H1,True,0.0,0.0


In [14]:
def next_city_drone(current_city):
    demand_copy = demand.copy(deep =True)
    demand_copy = demand_copy[demand_copy['Notation'] != current_city] # already tried, so removing
    demand_copy = demand_copy[demand_copy['Status of demand ( 1- met, 0- unmet)'] != 1]
    
    if len(demand_copy) == 0:
            return("ENDED-DRONE")
        
    demand_copy_before_loop = demand_copy.copy(deep=True)
    
    while True:
        if len(demand_copy) == 0:
            return("NOT FINISHED BUT NO PLACE FOR DRONE")
        highest_priority = demand_copy[demand_copy['Priority Factor'] == demand_copy['Priority Factor'].max()]
        selected_city = highest_priority.iloc[0, :]['Notation']
        distance = distance_dict[(current_city, selected_city)]
        #print("distance is ", distance)
        flight_time = distance/drone_speed
        #print("flight time is ", flight_time)
        if flight_time > 22.5:
            demand_copy = demand_copy[demand_copy['Notation'] != selected_city] # already tried, so removing
            continue
        elif flight_time <=22.5:
            blood_demand = highest_priority.iloc[0, :]['Average Demand on a given day/Kg']
            #print("blood demand is ", blood_demand)
            if blood_demand >= 4:
                demand_copy = demand_copy[demand_copy['Notation'] != selected_city] # already tried, so removing
                continue
            elif blood_demand < 4:
                break
    return selected_city
           
        
            
        

In [15]:
def next_city_ambulance(current_city):
    delivered_cities = list(demand[demand['Status of demand ( 1- met, 0- unmet)']==1]["Notation"])

    delivered_cities.append(current_city)  # can't go to current
    if len(demand) == len(delivered_cities):
        return "ENDED-AMBULANCE"
    distances_copy = distances.copy(deep = True)
    distances_row = distances_copy.loc[distances_copy['Unnamed: 0'] == current_city]
    distances_row = distances_row[distances_row.columns.drop(delivered_cities)]

    min_value = None
    min_index = None
    for index in range(len(distances_row.columns)):
        value = distances_row.iloc[:, index].values[0]
     
        if value == np.nan:
            continue
        if type(value) is str:
            continue
        if min_value is None or value < min_value:
            min_value = value
            min_index = index
    
    return(distances_row.columns[min_index])


In [16]:
together = True  # drone and ambulance
unmet_demand = True
no_work_for_drone = False
while unmet_demand:
    row = {
            'Drone start': drone_current_city,
             'Ambulance start': ambulance_current_city}
 
    
    
    
    if together and not no_work_for_drone:  # it's not turn for drone to refuel. 

        drone_next_city = next_city_drone(drone_current_city)
        if drone_next_city == 'ENDED-DRONE':
            print("All fulfilled.")
            break
        if drone_next_city == ("NOT FINISHED BUT NO PLACE FOR DRONE"):
            no_work_for_drone = True
            continue
            print("Not finished but could not find a place for drone")
            print("demand at this point is ")
        row['Drone End'] = drone_next_city
        row['Did Drone Deliver'] = True
        row['Drone Distance'] = distance_dict[(drone_current_city, drone_next_city)]
        demand.loc[demand['Notation'] ==drone_next_city, 'Status of demand ( 1- met, 0- unmet)'] = 1
        drone_current_city = drone_next_city
        ambulance_next_city = next_city_ambulance(ambulance_current_city)
        if ambulance_next_city == "ENDED-AMBULANCE":
            print("All fulfilled.")
            break
        row['Ambulance End'] = ambulance_next_city
        row['Did ambulance deliver'] = True
        row['Ambulance Distance'] = distance_dict[(ambulance_current_city, ambulance_next_city)]
        demand.loc[demand['Notation'] ==ambulance_next_city, 'Status of demand ( 1- met, 0- unmet)'] = 1
        ambulance_current_city = ambulance_next_city
        together = False
    elif not together and not no_work_for_drone:  # it's time to refuel drone
 
        ambulance_next_city = next_city_ambulance(ambulance_current_city)
        if ambulance_next_city == "ENDED-AMBULANCE":
            print("All fulfilled.")
            break
        row['Ambulance End'] = ambulance_next_city
        row['Drone End'] = ambulance_next_city
        row['Did ambulance deliver'] = True
        row['Did Drone Deliver'] = False
        row['Ambulance Distance'] = distance_dict[(ambulance_current_city, ambulance_next_city)]
        row['Drone Distance'] = distance_dict[(drone_current_city, ambulance_next_city)] # drone goes where ambulance goes
        demand.loc[demand['Notation'] ==ambulance_next_city, 'Status of demand ( 1- met, 0- unmet)'] = 1
        ambulance_current_city = ambulance_next_city
        drone_current_city = ambulance_next_city   # drone goes to where ambulance goes for refueling
        together = True
        
    elif no_work_for_drone:
        ambulance_next_city = next_city_ambulance(ambulance_current_city)
        if ambulance_next_city == "ENDED-AMBULANCE":
            print("All fulfilled.")
            break
        row['Ambulance End'] = ambulance_next_city
        row['Did ambulance deliver'] = True
        row['Did Drone Deliver'] = False
        row['Ambulance Distance'] = distance_dict[(ambulance_current_city, ambulance_next_city)]
        demand.loc[demand['Notation'] ==ambulance_next_city, 'Status of demand ( 1- met, 0- unmet)'] = 1
        ambulance_current_city = ambulance_next_city
        drone_next_city = "NA" 
        drone_current_city = "NA"
        together = False
        

    final_output = final_output.append(row, ignore_index = True)
  
#     print("Together or Not: ", together)
#     together = not together
    
#     print("ambulance next city is ", ambulance_next_city)
#     print("drone next city is ", drone_next_city)
   
        
        
    

C:\Users\manga\AppData\Local\Temp\ipykernel_25240\871219756.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_output = final_output.append(row, ignore_index = True)
C:\Users\manga\AppData\Local\Temp\ipykernel_25240\871219756.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_output = final_output.append(row, ignore_index = True)
C:\Users\manga\AppData\Local\Temp\ipykernel_25240\871219756.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_output = final_output.append(row, ignore_index = True)
C:\Users\manga\AppData\Local\Temp\ipykernel_25240\871219756.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_out

All fulfilled.


C:\Users\manga\AppData\Local\Temp\ipykernel_25240\871219756.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_output = final_output.append(row, ignore_index = True)
C:\Users\manga\AppData\Local\Temp\ipykernel_25240\871219756.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_output = final_output.append(row, ignore_index = True)
C:\Users\manga\AppData\Local\Temp\ipykernel_25240\871219756.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_output = final_output.append(row, ignore_index = True)
C:\Users\manga\AppData\Local\Temp\ipykernel_25240\871219756.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_out

In [14]:
final_output

,Drone start,Drone End,Did Drone Deliver,Ambulance start,Ambulance End,Did ambulance deliver,Ambulance Distance,Drone Distance
0,H1,H1,False,H1,H1,True,0.000000,0.000000
1,H1,H23,True,H1,H58,True,1.204945,15.682814
2,H23,H5,False,H58,H5,True,1.141721,15.663602
3,H5,H14,True,H5,H21,True,1.283599,10.589583
4,H14,H34,False,H21,H34,True,1.021381,4.104652
5,H34,H25,True,H34,H57,True,1.216853,15.154471
6,H25,H38,False,H57,H38,True,1.013490,13.023857
7,H38,H8,True,H38,H33,True,1.036333,16.176915
8,H8,H36,False,H33,H36,True,1.070908,8.823784
9,H36,H13,True,H36,H30,True,1.070235,11.458010


In [15]:
ls

 Volume in drive C is Windows
 Volume Serial Number is A466-585E

 Directory of C:\Users\manga

04/25/2023  10:31 PM    <DIR>          .
12/02/2022  12:45 AM    <DIR>          ..
12/10/2022  11:26 PM    <DIR>          .conda
04/25/2023  10:31 PM    <DIR>          .ipynb_checkpoints
12/02/2022  03:10 PM    <DIR>          .ipython
02/16/2023  01:18 AM    <DIR>          .julia
12/10/2022  11:22 PM    <DIR>          .jupyter
12/02/2022  03:10 PM    <DIR>          .matplotlib
12/14/2022  07:15 PM    <DIR>          Anaconda3
10/15/2022  08:55 AM           202,355 Computation Project - Time Series Forecasting.ipynb
10/16/2022  04:26 PM        15,238,469 Computational Project 1.1.ipynb
12/02/2022  12:18 AM    <DIR>          Contacts
04/23/2023  01:32 AM    <DIR>          Documents
04/25/2023  10:18 PM    <DIR>          Downloads
04/25/2023  10:12 PM            68,392 Drone-Data_Solutions_Operations-Research-1.xlsx
12/10/2022  11:34 PM             2,074 dtree_gini
12/14/2022  07:22 PM          

In [16]:
pwd

'C:\\Users\\manga'

In [17]:
final_output.to_excel("Jordan Blood Final Output.xlsx")

In [18]:
ls

 Volume in drive C is Windows
 Volume Serial Number is A466-585E

 Directory of C:\Users\manga

04/25/2023  10:31 PM    <DIR>          .
12/02/2022  12:45 AM    <DIR>          ..
12/10/2022  11:26 PM    <DIR>          .conda
04/25/2023  10:31 PM    <DIR>          .ipynb_checkpoints
12/02/2022  03:10 PM    <DIR>          .ipython
02/16/2023  01:18 AM    <DIR>          .julia
12/10/2022  11:22 PM    <DIR>          .jupyter
12/02/2022  03:10 PM    <DIR>          .matplotlib
12/14/2022  07:15 PM    <DIR>          Anaconda3
10/15/2022  08:55 AM           202,355 Computation Project - Time Series Forecasting.ipynb
10/16/2022  04:26 PM        15,238,469 Computational Project 1.1.ipynb
12/02/2022  12:18 AM    <DIR>          Contacts
04/23/2023  01:32 AM    <DIR>          Documents
04/25/2023  10:18 PM    <DIR>          Downloads
04/25/2023  10:12 PM            68,392 Drone-Data_Solutions_Operations-Research-1.xlsx
12/10/2022  11:34 PM             2,074 dtree_gini
12/14/2022  07:22 PM          